# ASR WOLOF IndabaX Senegal 2024




In [1]:
# Make sure you use Python 3.10
!python --version

Python 3.10.14


In [2]:
import numpy
numpy.version.version

'1.26.4'

In [3]:
!nvidia-smi


Mon Dec  2 13:25:47 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P0             30W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
#%%capture
!pip install transformers==4.35.2
!pip install speechbrain
!pip install datasets==2.15.0
!pip install evaluate==0.4.1
!pip install jiwer==3.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 54.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 50.2 MB/s eta 0:00:0000:01:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.0
    Uninstalling tokenizers-0.20.0:
      Successfully uninstalled tokenizers-0.20.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 24.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.2 MB/s eta 0:00:00
   

In [6]:
import numpy
numpy.version.version

'1.26.4'

In [10]:
import evaluate

/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [11]:
metric = evaluate.load("wer")

In [12]:
!pip install numpy==1.21.6

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 92.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
ucxx 0.39.1 requires libucx>=1.15.0, which is not installed.
albucore 0.0.17 requires numpy>=1.24, but you have numpy 1.21.6 which is incompatible.
albumentations 1.4.17 requires numpy>=1.24.4, but you have numpy 1.21.6 which is incompatible.
apache-beam 2.46.0 

In [13]:
import os
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset,concatenate_datasets, ClassLabel#set_caching_enabled,
import pandas as pd

import random
from IPython.display import display, HTML

import json
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2ForCTC,Wav2Vec2Processor,Trainer,TrainingArguments,Wav2Vec2FeatureExtractor

import re
#set_caching_enabled(False)

import soundfile as sf
import torchaudio


import IPython.display as ipd

import numpy as np
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from tqdm import tqdm
import torch

# Set environment variables
os.environ['WANDB_DISABLED '] = 'True'

import warnings
warnings.filterwarnings('ignore')



Replace your_hugging_face_key by your key to download the data from huggingface

In [ ]:
!python -c "from huggingface_hub.hf_api import HfFolder; HfFolder.save_token ('your_hugging_face_key')"

Ensure reproducibility

In [15]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False 
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(2018)

## Data


In [16]:
# Download data
dataset_train = load_dataset("IndabaxSenegal/asr-wolof-dataset", split = 'train')
dataset_train

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/13000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'duration', 'file_name', 'path', 'audio'],
    num_rows: 13000
})

In [17]:
dataset_train 

Dataset({
    features: ['text', 'duration', 'file_name', 'path', 'audio'],
    num_rows: 13000
})

In [18]:
# Download data
dataset_test = load_dataset("IndabaxSenegal/asr-wolof-dataset-test", split = 'test')
dataset_test

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['duration', 'file_name', 'path', 'audio'],
    num_rows: 2000
})

Split the data 90 percent for training and 10 percent for testing

In [19]:
dataset_train_split=dataset_train.train_test_split(test_size=0.1)
dataset_train_split

DatasetDict({
    train: Dataset({
        features: ['text', 'duration', 'file_name', 'path', 'audio'],
        num_rows: 11700
    })
    test: Dataset({
        features: ['text', 'duration', 'file_name', 'path', 'audio'],
        num_rows: 1300
    })
})

In [20]:
dataset_train_split=dataset_train_split.remove_columns(["duration", "file_name", "path"])

In [21]:
#file_name = dataset_test['file_name']
#

In [23]:
dataset_test=dataset_test.remove_columns(["duration", "file_name", "path"])

In [24]:
dataset_train_split["train"][0]

{'text': 'Nataal bii de maa ngi gis ab paa boo xam ne mi ngi taxaw toog ci ab fauteuil, ba noppi soliy weer am yu mu wékk ci ginnaawam waaw, mu sol am mbaxana mu weex, nekk ab paa boo xam ne nuru na ni paa bu màggat la daal bu màggat bu màggat a màggat.',
 'audio': {'path': '3562:1311.wav',
  'array': array([ 8.06385651e-06, -4.55416739e-07, -4.33530658e-06, ...,
          1.59557015e-01,  1.64027512e-01,  1.66705951e-01]),
  'sampling_rate': 48000}}

Normalize all audios to a sample rate of 16 000

In [25]:
from datasets import  Audio

In [26]:
dataset_train_split=dataset_train_split.cast_column("audio", Audio(sampling_rate=16_000))
dataset_train_split["train"][0]

{'text': 'Nataal bii de maa ngi gis ab paa boo xam ne mi ngi taxaw toog ci ab fauteuil, ba noppi soliy weer am yu mu wékk ci ginnaawam waaw, mu sol am mbaxana mu weex, nekk ab paa boo xam ne nuru na ni paa bu màggat la daal bu màggat bu màggat a màggat.',
 'audio': {'path': '3562:1311.wav',
  'array': array([ 1.67358667e-06, -2.53878534e-06, -1.57952309e-06, ...,
          1.09237760e-01,  1.67423606e-01,  0.00000000e+00]),
  'sampling_rate': 16000}}

In [27]:
dataset_test=dataset_test.cast_column("audio", Audio(sampling_rate=16_000))
dataset_test[0]

{'audio': {'path': 'wol_45412_722.854_724.791.wav',
  'array': array([-0.00033569, -0.00030518, -0.00064087, ..., -0.01861572,
         -0.01989746, -0.02392578]),
  'sampling_rate': 16000}}

In [28]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [29]:
dataset_train_split=dataset_train_split.map(remove_special_characters)

Map:   0%|          | 0/11700 [00:00<?, ? examples/s]

Map:   0%|          | 0/1300 [00:00<?, ? examples/s]

We build the vocabulary

In [30]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab_train_1 = dataset_train_split.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset_train_split.column_names["train"])
#vocab_test = dataset_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset_test.column_names)


Map:   0%|          | 0/11700 [00:00<?, ? examples/s]

Map:   0%|          | 0/1300 [00:00<?, ? examples/s]

In [31]:
vocab_train_1

DatasetDict({
    train: Dataset({
        features: ['vocab', 'all_text'],
        num_rows: 1
    })
    test: Dataset({
        features: ['vocab', 'all_text'],
        num_rows: 1
    })
})

In [32]:
vocab_list = list(set(vocab_train_1["train"]["vocab"][0]) |set(vocab_train_1["test"]["vocab"][0]) )

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

78

In [33]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=True, do_normalize=True, return_attention_mask=False)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [34]:
len(processor.tokenizer)

80

In [35]:
dataset_train_split

DatasetDict({
    train: Dataset({
        features: ['text', 'audio'],
        num_rows: 11700
    })
    test: Dataset({
        features: ['text', 'audio'],
        num_rows: 1300
    })
})

In [36]:
rand_int = random.randint(0, len(dataset_train_split["train"]))

print("Target text:", dataset_train_split["train"][rand_int]["text"])
print("Input array shape:", np.asarray(dataset_train_split["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", dataset_train_split["train"][rand_int]["audio"]["sampling_rate"])

Target text: bu boobaa perte  lay doon 
Input array shape: (24400,)
Sampling rate: 16000


### Data preparation

In [37]:


def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

def prepare_dataset_test(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    

    return batch



dataset_train_split_=dataset_train_split.map(prepare_dataset, remove_columns=dataset_train_split.column_names["train"], num_proc=4)
dataset_test_=dataset_test.map(prepare_dataset_test, remove_columns=dataset_test.column_names, num_proc=4)


Map (num_proc=4):   0%|          | 0/11700 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1300 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

In [38]:
dataset_train_split_

DatasetDict({
    train: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 11700
    })
    test: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 1300
    })
})

We keep audio with length less than 18 seconds because of computational resources

In [39]:
max_input_length_in_sec = 18.0
dataset_train_split_["train"] = dataset_train_split_["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])
dataset_train_split_["test"] = dataset_train_split_["test"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter:   0%|          | 0/11700 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1300 [00:00<?, ? examples/s]

In [40]:
dataset_train_split_

DatasetDict({
    train: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 10711
    })
    test: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 1194
    })
})

In [41]:
!nvidia-smi

Mon Dec  2 13:32:51 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P0             29W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [42]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None



    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=True,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=True,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )


        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [43]:
#data_collator=DataCollatorCTCWithPadding(processor=processor, padding="longest")
print(data_collator)

DataCollatorCTCWithPadding(processor=Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": true,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='', vocab_size=78, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	76: AddedToken("[UNK]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	77: AddedToken("[PAD]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	78: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	79: AddedToken("</s>", rstrip=

#### Evaluation function

In [44]:
import evaluate
import numpy as np

wer_eva=evaluate.load("wer")


def compute_metrics(pred):
    pred_logits=pred.predictions
    pred_ids=np.argmax(pred_logits, axis=-1)
    
    pred.label_ids[pred.label_ids==-100]=processor.tokenizer.pad_token_id
    
    pred_str=processor.batch_decode(pred_ids)
    label_str=processor.batch_decode(pred.label_ids, group_tokens=False)
    
    wer=wer_eva.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}



#### Model

In [45]:
model = Wav2Vec2ForCTC.from_pretrained(
     
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)



config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
model.freeze_feature_extractor()

In [47]:


print(model.config)

Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-xls-r-300m",
  "activation_dropout": 0.0,
  "adapter_attn_dim": null,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.0,
  "feat_quantizer_dropout": 0.0,
  "final_dro

#### Model arguments

In [48]:


from transformers import TrainingArguments


training_args = TrainingArguments(
  output_dir="./wav2vec-300m-max18-WF-epoch-16-batch-8",
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=16,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  load_best_model_at_end=True,
  metric_for_best_model="wer",
  greater_is_better=False,
)

trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train_split_["train"],
    eval_dataset=dataset_train_split_["test"],
    tokenizer=processor.feature_extractor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



In [49]:
!nvidia-smi

Mon Dec  2 13:33:06 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P0             36W /  250W |    1463MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [50]:
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1024"

In [51]:
trainer.train()#20 length wav2vec bqse batch 8 epoch 20

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Wer
500,6.270400,7.949221,1.000000
1000,3.190000,5.304725,1.000000
1500,2.735300,2.130722,0.872372
2000,2.058000,1.599802,0.699860
2500,1.758200,1.399437,0.673931
3000,1.643200,1.282198,0.582143
3500,1.489300,1.270384,0.557897
4000,1.501800,1.232424,0.537403
4500,1.386800,1.145862,0.525425
5000,1.315600,1.137554,0.507336


TrainOutput(global_step=21424, training_loss=1.2697633268941775, metrics={'train_runtime': 35375.3387, 'train_samples_per_second': 4.845, 'train_steps_per_second': 0.606, 'total_flos': 2.7237182155391726e+19, 'train_loss': 1.2697633268941775, 'epoch': 16.0})

In [52]:
import math

eval_results=trainer.evaluate()
print(f"Perplexity:{math.exp(eval_results['eval_loss']):2f}")


Perplexity:2.680796


In [53]:
processor.push_to_hub(repo_id='uriel/wav2vec-300m-max18-WF-epoch-16-batch-8')
kwargs={
    'model_name': 'wav2vec-300m-max18-WF-epoch-16-batch-8', 
    'finetuned_from': "wav2vec",
    'tasks': 'automatic-speech-recognition',
    'dataset':'IndabaxSenegal/asr-wolof-dataset'
}

#processor.push_to_hub(**kwargs)
trainer.push_to_hub(**kwargs)

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

events.out.tfevents.1733146698.0bc6ef8a4097.30.0:   0%|          | 0.00/26.1k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

events.out.tfevents.1733182236.0bc6ef8a4097.30.1:   0%|          | 0.00/412 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/uriel/wav2vec-300m-max18-WF-epoch-16-batch-8/commit/cbd31f535df1ae9cab052e6539cdb2ef18770b54', commit_message='End of training', commit_description='', oid='cbd31f535df1ae9cab052e6539cdb2ef18770b54', pr_url=None, repo_url=RepoUrl('https://huggingface.co/uriel/wav2vec-300m-max18-WF-epoch-16-batch-8', endpoint='https://huggingface.co', repo_type='model', repo_id='uriel/wav2vec-300m-max18-WF-epoch-16-batch-8'), pr_revision=None, pr_num=None)

#### Model evaluation and testing

In [54]:
df = pd.read_csv('/kaggle/input/test-data-indabax-senegal/test.csv')
df

,row_id,file_name
0,0,wol_45412_722.854_724.791.wav
1,1,wol_42412_1271.139_1287.175.wav
2,2,222:19.wav
3,3,wol_43712_247.955_257.824.wav
4,4,wol_43020_1944.283_1945.571.wav
...,...,...
1995,1995,wol_44511_572.052_580.019.wav
1996,1996,wol_44411_613.168_614.598.wav
1997,1997,wol_4812_479.203_486.47.wav
1998,1998,wol_43910_1025.695_1051.446.wav


In [55]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained('uriel/wav2vec-300m-max18-WF-epoch-16-batch-8').cuda()
processor_ = Wav2Vec2Processor.from_pretrained('uriel/wav2vec-300m-max18-WF-epoch-16-batch-8')

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/258 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/887 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [56]:


def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor_(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor_.as_target_processor():
        batch["labels"] = processor_(batch["text"]).input_ids
    return batch

def prepare_dataset_test(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor_(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    

    return batch



dataset_train_split_=dataset_train_split.map(prepare_dataset, remove_columns=dataset_train_split.column_names["train"], num_proc=4)
dataset_test_=dataset_test.map(prepare_dataset_test, remove_columns=dataset_test.column_names, num_proc=4)

max_input_length_in_sec = 18.0
dataset_train_split_["train"] = dataset_train_split_["train"].filter(lambda x: x < max_input_length_in_sec * processor_.feature_extractor.sampling_rate, input_columns=["input_length"])
dataset_train_split_["test"] = dataset_train_split_["test"].filter(lambda x: x < max_input_length_in_sec * processor_.feature_extractor.sampling_rate, input_columns=["input_length"])


Map (num_proc=4):   0%|          | 0/11700 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1300 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11700 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1300 [00:00<?, ? examples/s]

In [57]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor_.batch_decode(pred_ids)[0]
  batch["text"] = processor_.decode(batch["labels"], group_tokens=False)
  
  return batch

In [58]:
results = dataset_train_split_["test"].map(map_to_result, remove_columns=dataset_train_split_["test"].column_names)

Parameter 'function'=<function map_to_result at 0x7f9b92509bd0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1194 [00:00<?, ? examples/s]

In [59]:
wer_eva=evaluate.load("wer")

In [60]:
print("Test WER: {:.3f}".format(wer_eva.compute(predictions=results["pred_str"], references=results["text"]))) #Test WER: 0.440 Test WER: 0.388 Test WER: 0.376

Test WER: 0.343


In [61]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [62]:
show_random_elements(results)

,pred_str,text
0,boo jëlee garab gi su yaayam mooy suuf si,boo jëlee garab gi su yaayam mooy suuf si
1,wala len wala len wala mais toujour rale dina am mesure bi nga xamante ne bidinoo ko génnee,wala leneen wala leneen wala leneen mais toujours daal dina am mesure bi nga xamante ni bii dina ko génne
2,am na ay nombres yoo xam ne nii li muy wane mooy e kuréel ga nga xam ne nii mooy moo jënd côde bare am nombre yoo xam ne dafay wane produits yi ba ak fa muy fa mu jógee maanaam benn su() sufre bu ne a ak ak kii ak sémbali am na lu muy am na lu muy tekki,am na ay nombre yoo xam ne li muy wane mooy e kuréel ga nga xam ne nii mooy moo jënd code barre bi am nombres yoo xam ne dafay wane produit ba ak fa mu fa mu jógee maanaam benn chiffre bu ne ak ak ak kii ak symbole yi am na lu muy am na lu muy tekki
3,yooyu yëpp booy tabax xeetu bor,yooyu yëpp booy tabax xeetu bor
4,li jëti jëf ak jiwu ak da daf ci rënd bas la ca topp a la ci topp ak asakaa yaak ex ñu la ca top ak la ca topp kon ci foofu la ñu toll,lijjanti ci ag jiwu ak nga def ci rënd baag la ca topp ak la ca topp ak asaka yaag ak la ca topp ak la ca topp kon ci foofu la ñu toll
5,a aa da a ee a a baaa,ci normalement seize mille nga waroon fay
6,parce que bu fekkee gerte gi egna maturité bu tawee boo ko déeqiwul rek dafay rugine mi dafay dafay dafay saxaat te bu saxee dafay fa posé problème bu fekkee fekkoon ne la nga déqia paree te fekkee da nga ko ngoron jo tuguloo ko naaf,parce que bu_ fekkee gerte gi egg na à maturité bu tawee boo ko deqiwul dafay regermer waat dafay dafay saxaat te bu saxee dafay dafay poser problème bu fekkee fekkoon na la nga deqi ba pare te fekk da nga ko ngaroon rek jotaguloo ko naaf
7,mun na law encore ci biire région bi,sàmmkat yi ba information bi war
8,suñ paree ñu accompagner leen ñu dem liggéeyi jëriñoo,suñ paree ñu accompagner leen ñu dem liggéeyi waa jëriñoo
9,tey kooku genre engrais la ah si la photo cintaie parce que tey jant bi dafa am rôle bi nga xam ne daf koy jouér si plante bi xam nga mum mooy accumuler matière organique yi pour plante bi mun a développer,tey kooku genre engrais la ah si la photosynthèse parce fa que tey jant bi dafa am rôle bi nga xam ni daf koy joué si plante bi xam nga mooy accumulé matières organique yi pour plante bi mën a développé


In [63]:
def predict(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor_.batch_decode(pred_ids)[0]
  #batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [64]:

results = dataset_test_.map(predict, remove_columns=dataset_test_.column_names)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [65]:
prediction = []
for i in range(len(results)):
    prediction.append(results['pred_str'][i])
    

In [66]:
df["text"] = prediction
df


,row_id,file_name,text
0,0,wol_45412_722.854_724.791.wav,day absorber pesticiée bi fu mu mën a ne
1,1,wol_42412_1271.139_1287.175.wav,dama koy baamatuwaat tey encore jël leen sama...
2,2,222:19.wav,nataal bii nag ñu ngi ci gis ak daamaar goo xa...
3,3,wol_43712_247.955_257.824.wav,waaw defaraat ku bam ba m kii yooyu yëpp ay mb...
4,4,wol_43020_1944.283_1945.571.wav,aau a a
...,...,...,...
1995,1995,wol_44511_572.052_580.019.wav,e ancar mi nekk e kuréel boo xamante ne état ...
1996,1996,wol_44411_613.168_614.598.wav,aut suuf pour mun béy
1997,1997,wol_4812_479.203_486.47.wav,ndax ñe gi lay samoromi jigéen yi sax presque ...
1998,1998,wol_43910_1025.695_1051.446.wav,ah man nga ko def e ci ay bidons c wus donc ...


We have some missing elements in the text column, we will replace them by random elements

In [67]:
nan_rows = df[df['text'].isna()]
nan_rows

,row_id,file_name,text


In [68]:
nan_rows = df[df['text'].isnull()]
nan_rows

,row_id,file_name,text


In [70]:
df.iloc[220,] 

row_id                                  220
file_name    wol_4410_1781.131_1781.656.wav
text                                       
Name: 220, dtype: object

In [71]:
df.iloc[728,] 

row_id                                   728
file_name    wol_43612_1370.469_1371.086.wav
text                                        
Name: 728, dtype: object

In [72]:
df.iloc[220,2] = "ah"
df.iloc[728,2] = "ah"
df.iloc[1256,2] = "ah"
df.iloc[1618,2] = "ah"

In [73]:
df.iloc[220,] 

row_id                                  220
file_name    wol_4410_1781.131_1781.656.wav
text                                     ah
Name: 220, dtype: object

Final submission

In [69]:
df[["row_id","text"]].to_csv("submission_NEW8.csv",index=False)

## End